# 지하철역 소재와 서울시 구/행정동 연결

### 1. 데이터 준비

In [46]:
# 1-1. 필요 라이브러리 불러오기

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

# 한글 폰트 설정
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [47]:
# 1-2. 데이터 파일 불러오기
# 행정동 경계파일 불러오기
dong_gdf = gpd.read_file\
    ('../4.Public_transportation/BND_ADM_DONG_PG_Adong/BND_ADM_DONG_PG.shp',
     encoding='euc-kr')

# 구 경계파일 불러오기
gu_gdf = gpd.read_file\
    ('../4.Public_transportation/LARD_ADM_SECT_SGG_gu/LARD_ADM_SECT_SGG_11_202405.shp',
     encoding='cp949')

# 구별 행정동 목록파일 불러오기
gu_dong_df = pd.read_csv\
    ('../4.Public_transportation/Gu_and_Administrative_dong.csv', encoding='cp949')

# 지하철역 데이터 불러오기
station_df = pd.read_csv\
    ('../4.Public_transportation/subway_latitude_and_longitude.csv', encoding='cp949')

### 2. 불러온 데이터 확인

In [48]:
print("=== 데이터 현황 ===")
print("\n[구 경계 데이터]")
print("- 데이터 크기:", gu_gdf.shape)
print("- 컬럼 목록:", gu_gdf.columns.tolist())
print("- 구 목록:", sorted(gu_gdf['SGG_NM'].tolist()))
print("- 좌표계:", gu_gdf.crs)

print("\n[행정동 경계 데이터]")
print("- 데이터 크기:", dong_gdf.shape)
print("- 컬럼 목록:", dong_gdf.columns.tolist())
print("- 좌표계:", dong_gdf.crs)

print("\n[지하철역 데이터]")
print("- 데이터 크기:", station_df.shape)
print("- 컬럼 목록:", station_df.columns.tolist())
print("- 호선 현황:")
print(station_df['line'].value_counts().sort_index())


=== 데이터 현황 ===

[구 경계 데이터]
- 데이터 크기: (25, 5)
- 컬럼 목록: ['ADM_SECT_C', 'SGG_NM', 'SGG_OID', 'COL_ADM_SE', 'geometry']
- 구 목록: ['서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구', '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구', '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구', '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 송파구', '서울특별시 양천구', '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구', '서울특별시 중랑구']
- 좌표계: PROJCS["Korea_2000_Korea_Central_Belt_2010",GEOGCS["GCS_Korea_2000",DATUM["Korean_Geodetic_Datum_2002",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6737"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",38],PARAMETER["central_meridian",127],PARAMETER["scale_factor",1],PARAMETER["false_easting",200000],PARAMETER["false_northing",600000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]



### 3. 데이터 전처리

In [49]:
# 3.1 좌표계 통일 (모두 EPSG:4326으로 변환)
dong_gdf = dong_gdf.to_crs("EPSG:4326")
gu_gdf = gu_gdf.to_crs("EPSG:4326")

# 3.2 지하철역 데이터를 GeoDataFrame으로 변환
station_geometry = [Point(xy) for xy in zip(station_df['longitude'], station_df['latitude'])]
station_gdf = gpd.GeoDataFrame(
    station_df, 
    geometry=station_geometry,
    crs="EPSG:4326"
)

print("\n=== 전처리 결과 ===")
print("- 행정동 좌표계:", dong_gdf.crs)
print("- 구 좌표계:", gu_gdf.crs)
print("- 지하철역 좌표계:", station_gdf.crs)


=== 전처리 결과 ===
- 행정동 좌표계: EPSG:4326
- 구 좌표계: EPSG:4326
- 지하철역 좌표계: EPSG:4326


### 4. 공간 분석 작업

In [50]:
# 4.1 역과 구 매칭
stations_with_gu = gpd.sjoin(station_gdf, gu_gdf, how='left', predicate='within')

# 4.2 역과 동 매칭
stations_with_dong = gpd.sjoin(station_gdf, dong_gdf, how='left', predicate='within')

# 4.3 최종 데이터프레임 생성
result_df = pd.DataFrame({
    'station_name': stations_with_gu['name'],
    'line': stations_with_gu['line'],
    'gu': stations_with_gu['SGG_NM'],
    'dong': stations_with_dong['ADM_NM']
})

# 4.4 서울시 외 지역 제거
result_df = result_df.dropna(subset=['gu', 'dong'])
result_df = result_df.sort_values(['gu', 'dong', 'line', 'station_name'])
result_df.info()

# 4.5 분석 결과 출력
print(f"서울시 지하철역 수: {len(result_df)}개")

gu_station_count = result_df.groupby('gu').size().sort_values(ascending=False)
print("\n구별 지하철역 수:")
print(gu_station_count)
result_df.head()
result_df['gu'].nunique(), result_df['dong'].nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 407 entries, 638 to 633
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   station_name  407 non-null    object
 1   line          407 non-null    object
 2   gu            407 non-null    object
 3   dong          407 non-null    object
dtypes: object(4)
memory usage: 15.9+ KB
서울시 지하철역 수: 407개

구별 지하철역 수:
gu
서울특별시 강남구     37
서울특별시 송파구     30
서울특별시 중구      26
서울특별시 마포구     24
서울특별시 영등포구    23
서울특별시 강서구     21
서울특별시 서초구     20
서울특별시 성동구     19
서울특별시 용산구     18
서울특별시 동작구     18
서울특별시 은평구     16
서울특별시 강동구     15
서울특별시 종로구     14
서울특별시 노원구     14
서울특별시 성북구     14
서울특별시 중랑구     13
서울특별시 구로구     13
서울특별시 광진구     11
서울특별시 동대문구    11
서울특별시 강북구     11
서울특별시 관악구     10
서울특별시 서대문구     9
서울특별시 양천구      8
서울특별시 도봉구      8
서울특별시 금천구      4
dtype: int64


(25, 233)

### 5. 서울시 모든 구별 행정동과 매칭

In [51]:
# 5-1. 최종 데이터프레임 가공
# '서울특별시'와 '구' 이름 분리
result_df['city'] = result_df['gu'].str.split(' ').str[0]
result_df['gu'] = result_df['gu'].str.split(' ').str[1]

In [52]:
# 5-2. 최종 데이터프레임 컬럼 순서 변경
result_df = result_df[['station_name', 'line', 'city', 'gu', 'dong']]
result_df['gu_dong'] = result_df['gu'] + '_' + result_df['dong']
result_df.head()

,station_name,line,city,gu,dong,gu_dong
638,개포동,분당선,서울특별시,강남구,개포2동,강남구_개포2동
639,구룡,분당선,서울특별시,강남구,개포2동,강남구_개포2동
637,대모산입구,분당선,서울특별시,강남구,개포2동,강남구_개포2동
699,대청,3호선,서울특별시,강남구,개포3동,강남구_개포3동
709,신사,3호선,서울특별시,강남구,논현1동,강남구_논현1동


In [53]:
# 5-3. gu_dong_df 전처리
# gu_dong_df = gu_dong_df[['gu', 'legal_dong']]
gu_dong_df.columns = ['gu', 'dong']
gu_dong_df['gu_dong'] = gu_dong_df['gu'] + '_' + gu_dong_df['dong']
gu_dong_df = gu_dong_df[['gu', 'dong', 'gu_dong']]

# 5-4. 'gu'와 'dong' 조합으로 고유값 세기
unique_gu_dong_count = gu_dong_df['gu_dong'].drop_duplicates().shape[0]
print(f"구-동 조합의 고유값 개수: {unique_gu_dong_count}")

구-동 조합의 고유값 개수: 426


In [54]:
# 5-5. gu_dong_df와 result_df를 dong을 기준으로 left outer join
merged_df = gu_dong_df.merge(result_df, 
                            on='gu_dong', 
                            how='left')
merged_df.info()
# 5-6. station_name과 line이 없는 동은 NaN으로 자동 채워짐
# 필요한 컬럼만 선택하고 정렬
final_df = merged_df[['station_name', 'line', 'gu_x', 'dong_x', 'gu_dong']].sort_values('gu_dong')
final_df.columns = ['station_name', 'line', 'gu', 'dong', 'gu_dong']
final_df.info()

# 5-7. 'gu'와 'dong' 조합으로 고유값 세기
unique_gu_dong_count = final_df['gu_dong'].drop_duplicates().shape[0]
print(f"구-동 조합의 고유값 개수: {unique_gu_dong_count}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   gu_x          598 non-null    object
 1   dong_x        598 non-null    object
 2   gu_dong       598 non-null    object
 3   station_name  402 non-null    object
 4   line          402 non-null    object
 5   city          402 non-null    object
 6   gu_y          402 non-null    object
 7   dong_y        402 non-null    object
dtypes: object(8)
memory usage: 37.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 598 entries, 33 to 589
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   station_name  402 non-null    object
 1   line          402 non-null    object
 2   gu            598 non-null    object
 3   dong          598 non-null    object
 4   gu_dong       598 non-null    object
dtypes: object(5)
memory usage: 28.0+ KB


### 6. 행정동별 지하철 유무 구분

In [55]:
# 6-1. 지하철이 있는 행정동
yes_station = result_df
print( yes_station['gu_dong'].nunique() )

# 6-2. 지하철이 없는 행정동
# 빈 데이터프레임 생성 (final_df와 동일한 컬럼을 사용)
no_station = pd.DataFrame(columns=final_df.columns)
# station_name 열이 NaN인 행 필터링
na_rows = final_df[final_df['station_name'].isna()]
# 필터링된 행들을 not_station에 추가
no_station = pd.concat([no_station, na_rows], ignore_index=True)
no_station.info()
print(no_station['gu_dong'].nunique() )

234
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   station_name  0 non-null      object
 1   line          0 non-null      object
 2   gu            196 non-null    object
 3   dong          196 non-null    object
 4   gu_dong       196 non-null    object
dtypes: object(5)
memory usage: 7.8+ KB
196


### 7. 결과 저장

In [56]:
# 지하철역이 있는 행정동
yes_station.to_csv('수연씨선물/seoul_subway_locations_yes_A.csv', index=False, encoding='cp949')

# 지하철역이 없는 행정동
no_station.to_csv('수연씨선물/seoul_subway_locations_no_A.csv', index=False, encoding='cp949')

# 둘을 종합한 행정동
final_df.to_csv('수연씨선물/seoul_subway_locations_all_A.csv', index=False, encoding='cp949')

In [57]:
# 상세 위치 정보 출력
print("\n=== 구별 지하철역 상세 목록 ===")
for gu in sorted(result_df['gu'].unique()):
    print(f"\n[{gu}]")
    gu_data = result_df[result_df['gu'] == gu].sort_values(['dong', 'line'])
    for _, row in gu_data.iterrows():
        print(f"- {row['station_name']}({row['line']}): {row['dong']}")


=== 구별 지하철역 상세 목록 ===

[강남구]
- 개포동(분당선): 개포2동
- 구룡(분당선): 개포2동
- 대모산입구(분당선): 개포2동
- 대청(3호선): 개포3동
- 신사(3호선): 논현1동
- 논현(7호선): 논현1동
- 논현(신분당선(연장2)): 논현1동
- 신사(신분당선(연장2)): 논현1동
- 강남구청(7호선): 논현2동
- 학동(7호선): 논현2동
- 강남구청(분당선): 논현2동
- 선정릉(분당선): 논현2동
- 도곡(3호선): 대치1동
- 삼성(무역센터)(2호선): 대치2동
- 대치(3호선): 대치2동
- 학여울(3호선): 대치2동
- 삼성(수도권 광역급행철도): 대치2동
- 매봉(3호선): 도곡1동
- 도곡(분당선): 도곡2동
- 한티(분당선): 도곡2동
- 청담(7호선): 삼성1동
- 봉은사(9호선(연장)): 삼성1동
- 삼성중앙(9호선(연장)): 삼성2동
- 선정릉(9호선(연장)): 삼성2동
- 수서(3호선): 수서동
- 수서(분당선): 수서동
- 수서(수도권 광역급행철도): 수서동
- 압구정(3호선): 압구정동
- 압구정로데오(분당선): 압구정동
- 강남(2호선): 역삼1동
- 선릉(2호선): 역삼1동
- 역삼(2호선): 역삼1동
- 신논현(9호선): 역삼1동
- 언주(9호선(연장)): 역삼1동
- 선릉(분당선): 역삼1동
- 신논현(신분당선(연장2)): 역삼1동
- 일원(3호선): 일원본동

[강동구]
- 강일(5호선): 강일동
- 강일(5호선): 강일동
- 길동(5호선): 길동
- 둔촌동(5호선): 둔촌1동
- 둔촌오륜(9호선(연장)): 둔촌1동
- 중앙보훈병원(9호선(연장)): 둔촌2동
- 명일(5호선): 명일1동
- 고덕(5호선): 명일2동
- 상일동(5호선): 상일1동
- 강동구청(8호선): 성내1동
- 강동(5호선): 성내2동
- 천호(풍납토성)(8호선): 성내2동
- 암사(8호선): 암사2동
- 암사역사공원(8호선): 암사3동
- 굽은다리(강동구민회관앞)(5호선): 천호1동

[강북구]
- 미아(서울사이버대학)(4호선

### 8. 구별/동별 수 집계

In [58]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 598 entries, 33 to 589
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   station_name  402 non-null    object
 1   line          402 non-null    object
 2   gu            598 non-null    object
 3   dong          598 non-null    object
 4   gu_dong       598 non-null    object
dtypes: object(5)
memory usage: 28.0+ KB


In [59]:
gu_dong_df.info()
print(gu_dong_df.to_string())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gu       426 non-null    object
 1   dong     426 non-null    object
 2   gu_dong  426 non-null    object
dtypes: object(3)
memory usage: 10.1+ KB
       gu         dong          gu_dong
0     강남구          신사동          강남구_신사동
1     강남구         논현1동         강남구_논현1동
2     강남구         논현2동         강남구_논현2동
3     강남구         압구정동         강남구_압구정동
4     강남구          청담동          강남구_청담동
5     강남구         삼성1동         강남구_삼성1동
6     강남구         삼성2동         강남구_삼성2동
7     강남구         대치1동         강남구_대치1동
8     강남구         대치2동         강남구_대치2동
9     강남구         대치4동         강남구_대치4동
10    강남구         역삼1동         강남구_역삼1동
11    강남구         역삼2동         강남구_역삼2동
12    강남구         도곡1동         강남구_도곡1동
13    강남구         도곡2동         강남구_도곡2동
14    강남구         개포1동         강남구_개포1동
15    강남구         개포2동        

In [60]:
# 8-1. 집계를 위한 Dataframe 만들기
grouped = final_df.groupby(['gu', 'dong', 'gu_dong'])['station_name'].count().reset_index()
grouped.columns = ['gu', 'dong', 'gu_dong', 'station_count']
# 모든 행정동과 병합 (outer join)
all_station_dong = gu_dong_df.merge(grouped, on=['gu', 'dong', 'gu_dong'], how='left')

# 지하철역이 없는 동은 station_count를 0으로 설정
all_station_dong['station_count'] = all_station_dong['station_count'].fillna(0).astype(int)

# 결과 확인
print(all_station_dong)


      gu  dong   gu_dong  station_count
0    강남구   신사동   강남구_신사동              0
1    강남구  논현1동  강남구_논현1동              4
2    강남구  논현2동  강남구_논현2동              4
3    강남구  압구정동  강남구_압구정동              2
4    강남구   청담동   강남구_청담동              0
..   ...   ...       ...            ...
421  중랑구   묵2동   중랑구_묵2동              1
422  중랑구  망우본동  중랑구_망우본동              3
423  중랑구  망우3동  중랑구_망우3동              0
424  중랑구  신내1동  중랑구_신내1동              1
425  중랑구  신내2동  중랑구_신내2동              0

[426 rows x 4 columns]


In [61]:
# 8-2. 구별 지하철역 수 집계
# 8-2-1. 모든 구별 지하철역 수
station_gu_all = all_station_dong.groupby('gu')['station_count'].sum().reset_index()
# 8-2-2. 지하철역이 있는 구별 지하철역 수
station_gu_yes = station_gu_all[station_gu_all['station_count'] > 0]
# 8-2-3. 지하철역이 없는 구 목록
station_gu_no = station_gu_all[station_gu_all['station_count'] == 0]

station_gu_all.info(), station_gu_yes.info(), station_gu_no.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gu             25 non-null     object
 1   station_count  25 non-null     int32 
dtypes: int32(1), object(1)
memory usage: 428.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gu             25 non-null     object
 1   station_count  25 non-null     int32 
dtypes: int32(1), object(1)
memory usage: 428.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gu             0 non-null      object
 1   station_count  0 non-null      int32 
dtypes: int32(1), object(1)
memory usage: 0.0+ bytes


(None, None, None)

In [62]:
# 8-3. 동별 지하철역 수 집계
# 8-3-1. 모든 동별 지하철역 수
station_dong_all = all_station_dong.groupby(['gu', 'dong', 'gu_dong'])['station_count'].sum().reset_index()
# 8-3-2. 지하철역이 있는 동별 지하철역 수
station_dong_yes = station_dong_all[station_dong_all['station_count'] > 0]
# 8-3-3. 지하철역이 없는 동 목록
station_dong_no = station_dong_all[station_dong_all['station_count'] == 0]

station_dong_all.info(), station_dong_yes.info(), station_dong_no.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gu             426 non-null    object
 1   dong           426 non-null    object
 2   gu_dong        426 non-null    object
 3   station_count  426 non-null    int32 
dtypes: int32(1), object(3)
memory usage: 11.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 230 entries, 1 to 425
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gu             230 non-null    object
 1   dong           230 non-null    object
 2   gu_dong        230 non-null    object
 3   station_count  230 non-null    int32 
dtypes: int32(1), object(3)
memory usage: 8.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 196 entries, 0 to 424
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  --

(None, None, None)

### 8. 집계 파일 저장

In [63]:
station_gu_all.to_csv('수연씨선물/4_station_gu_all_Adong.csv', index = False)
station_gu_yes.to_csv('수연씨선물/4_station_gu_yes_Adong.csv', index = False)
station_gu_no.to_csv('수연씨선물/4_station_gu_no_Adong.csv', index = False)
station_dong_all.to_csv('수연씨선물/4_station_dong_all_Adong.csv', index = False)
station_dong_yes.to_csv('수연씨선물/4_station_dong_yes_Adong.csv', index = False)
station_dong_no.to_csv('수연씨선물/4_station_dong_no_Adong.csv', index = False)